# CogniAble
## Integration of Retrieval Augmented Generation (RAG) with Open Source LLM and LangChain for Autism Intervention Research.
> - By - Nandan Hemanth
> - Github repo: - https://github.com/NandanHemanth/Multimex-ingestion 
> - Resume - https://drive.google.com/file/d/1IB6X_G7mPwvzv1M8I0QiV099_02bylI2/view?usp=sharing 
---
1. Installing all the necessary libraries for this task

In [3]:
%pip install langchain
%pip install torch
%pip install sentence_transformers
%pip install faiss-cpu
%pip install huggingface-hub
%pip install pypdf
%pip install tiktoken
%pip -q install accelerate
%pip install llama-cpu-python
%pip -q install git+https://github.com/huggingface/transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement llama-cpu-python (from versions: none)
ERROR: No matching distribution found for llama-cpu-python


Note: you may need to restart the kernel to use updated packages.


2. Importing all the necessary for RAG with open-source LLM & langchain

In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader